In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import sqlite3

%run "../../apikey.py" #outside actual git repo, make sure to add file/replace with own apikey
VERSION = "v2.2"
OUTPUT_FORMAT = "json"
API_KEY = keys['CUMTD']

In [2]:
def cumtdRequestUrl(methodname, other_args={}, version=VERSION, output=OUTPUT_FORMAT, key=API_KEY):
    rooturl = "https://developer.cumtd.com/api/{v}/{f}/".format(v=VERSION, f=OUTPUT_FORMAT)
    url = rooturl + methodname + "?key={}".format(key)
    for param, value in other_args.items():
        url = url + "&{}={}".format(param, value)
    return url

In [3]:
# check API usage, cuz why not
requests.get(cumtdRequestUrl("getapiusage")).json()

{'time': '2018-11-07T07:48:26-06:00',
 'new_changeset': True,
 'status': {'code': 200, 'msg': 'ok'},
 'rqst': {'method': 'GetApiUsage', 'params': {}},
 'days': []}

In [4]:
# get all stops
r = requests.get(cumtdRequestUrl("getstops"))
json = r.json()

In [5]:
# arrange into csv
def generateAllStopsCsv(filename):
    r = requests.get(cumtdRequestUrl("getstops"))
    json = r.json()
    
    stops = {'stop_id': [], 'stop_name': [], 'code': [], 'distance': [], 'specific_stop_code': [], 'specific_stop_stop_id': [], 'specific_stop_stop_lat': [], 'specific_stop_stop_lon': [], 'specific_stop_stop_name': []}

    for stop in json['stops']:
        for key in stop.keys():
            if key in ['stop_id', 'stop_name', 'code', 'distance']:
                stops[key].append(stop[key])
            elif key == 'stop_points': pass
        for key in ['specific_stop_code', 'specific_stop_stop_id', 'specific_stop_stop_lat', 'specific_stop_stop_lon', 'specific_stop_stop_name']:
            stops[key].append('')
        for specific_stop in stop['stop_points']:
            for key in specific_stop.keys():
                if key in ['code', 'stop_id', 'stop_lat', 'stop_lon', 'stop_name']:
                    stops['specific_stop_' + key].append(specific_stop[key])
            for key in ['stop_id', 'stop_name', 'code', 'distance']:
                stops[key].append('')

    pd.DataFrame(stops).to_csv(filename, index=False)
    
# run once per folder
# generateAllStopsCsv('all_stops.csv')

In [6]:
def nameToStopId(name):
    stops = pd.read_csv('all_stops.csv')
    if name in list(stops['stop_name']):
        return stops[stops['stop_name'] == name].iloc[0]['stop_id']
    elif name in list(stops['specific_stop_stop_name']):
        return stops[stops['specific_stop_stop_name'] == name].iloc[0]['specific_stop_stop_id']
    else:
        return None
    
nameToStopId('U.S. 150 and Dale')

'150DALE'

In [7]:
STOP = nameToStopId('Goodwin & Main (SE Corner)')
r = requests.get(cumtdRequestUrl("getdeparturesbystop", {'stop_id': STOP, 'pt': 60}))
json = r.json()
print(json)

print(json['status']['msg'])

{'time': '2018-11-07T07:48:29-06:00', 'new_changeset': True, 'status': {'code': 200, 'msg': 'ok'}, 'rqst': {'method': 'GetDeparturesByStop', 'params': {'pt': 60, 'stop_id': 'GWNMN:2'}}, 'departures': [{'stop_id': 'GWNMN:2', 'headsign': '22N Illini', 'route': {'route_color': '5a1d5a', 'route_id': 'ILLINI', 'route_long_name': 'Illini', 'route_short_name': '22', 'route_text_color': 'ffffff'}, 'trip': {'trip_id': '[@7.0.41200832@][1][1238430123625]/4__I8_UIMF', 'trip_headsign': 'Lincoln & Killarney', 'route_id': 'ILLINI', 'block_id': 'I8 UIMF', 'direction': 'North', 'service_id': 'I8 UIMF', 'shape_id': '22N ILLINI 10'}, 'vehicle_id': '0109', 'origin': {'stop_id': 'PAR:2'}, 'destination': {'stop_id': 'LNCLNKLRNY:1'}, 'is_monitored': True, 'is_scheduled': True, 'is_istop': True, 'scheduled': '2018-11-07T07:52:24-06:00', 'expected': '2018-11-07T07:53:09-06:00', 'expected_mins': 5, 'location': {'lat': 40.10545, 'lon': -88.229}}, {'stop_id': 'GWNMN:2', 'headsign': '22N Illini', 'route': {'route

In [8]:
trip_id = []
scheduled = []
diffs = []
is_scheduled = []

for departure in json['departures']:
    trip_id.append(departure['trip']['trip_id'])
    scheduled_time = datetime.fromisoformat(departure['scheduled'])
    diff = (datetime.fromisoformat(departure['expected']) - scheduled_time).seconds
    
    scheduled.append(str(scheduled_time)) 
    diffs.append(diff)
    is_scheduled.append(departure['is_scheduled'])
    
df = pd.DataFrame({'trip_id': trip_id, 'scheduled_time': scheduled, 'diff': diffs, 'is_scheduled':is_scheduled})
df

,trip_id,scheduled_time,diff,is_scheduled
0,[@7.0.41200832@][1][1238430123625]/4__I8_UIMF,2018-11-07 07:52:24-06:00,45,True
1,[@7.0.41200832@][1][1238430123625]/5__I1UIMF,2018-11-07 08:02:24-06:00,94,True
2,[@7.0.41200832@][1][1238430123625]/6__I4_UIMF,2018-11-07 08:12:24-06:00,0,True
3,[@7.0.41200832@][1][1238430123625]/7__I6_UIMF,2018-11-07 08:22:24-06:00,0,True
4,[@7.0.41200832@][1][1238430123625]/8__I2_UIMTH,2018-11-07 08:32:24-06:00,0,True
5,[@7.0.41200832@][1][1238430123625]/9__I3_UIMTH,2018-11-07 08:42:24-06:00,0,True


In [9]:
stop_times_all = pd.read_csv('../../google_transit/stop_times.txt')
stop_times_all.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,R1SATPO__R1SA,07:08:00,07:08:00,DEPOT:1,0,Champaign Meijer,0,0
1,R1SATPO__R1SA,07:13:00,07:13:00,IU:2,2,Champaign Meijer,0,0
2,20RED001__R1SA,07:13:00,07:13:00,IU:2,0,Champaign Meijer,0,0
3,20RED001__R1SA,07:14:00,07:14:00,WRTHLY:2,1,NaN,0,0
4,20RED001__R1SA,07:15:30,07:15:30,WHTWRT:2,2,NaN,0,0


In [10]:
trip_id = '[@2.0.80548152@][12][1425572286750]/26__BB2_MF'
departure_time = '15:29:00'
expected_time = '15:29:06'
day = '10/30'
# stop_times_all.loc[(stop_times_all['trip_id'] == trip_id) & (stop_times_all['departure_time'] == departure_time),day] = expected_time
# stop_times_all[(stop_times_all['trip_id'] == trip_id) & (stop_times_all['departure_time'] == departure_time)]

In [11]:
stop_times_all[stop_times_all['trip_id'] == '[@7.0.41200832@][1][1238430123625]/44__I8_UIMF'].head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
109938,[@7.0.41200832@][1][1238430123625]/44__I8_UIMF,14:08:00,14:08:00,PAR:2,0,Lincoln & Killarney,0,0
109939,[@7.0.41200832@][1][1238430123625]/44__I8_UIMF,14:08:50,14:08:50,PAMD:2,1,NaN,0,0
109940,[@7.0.41200832@][1][1238430123625]/44__I8_UIMF,14:10:30,14:10:30,LAR:2,2,NaN,0,0
109941,[@7.0.41200832@][1][1238430123625]/44__I8_UIMF,14:11:20,14:11:20,LNCLNOR:2,3,NaN,0,0
109942,[@7.0.41200832@][1][1238430123625]/44__I8_UIMF,14:12:10,14:12:10,LNCLNIL:8,4,NaN,0,0


In [53]:
def cumtd_csv_to_sqlite(csv_file, table_name, sqlite_file):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()

    columns = {'trip_id': 'VARCHAR(60) NOT NULL', 'arrival_time': 'VARCHAR(8) NOT NULL', 'stop_id': 'VARCHAR(17) NOT NULL', 'stop_sequence': 'INTEGER', 'stop_headsign': 'VARCAHAR(36) NOT NULL', 'arrival_id': 'VARCHAR(63) NOT NULL PRIMARY KEY'}
    create_table_str = "CREATE TABLE IF NOT EXISTS {} (".format(table_name)
    for colname, coltype in columns.items():
        create_table_str += colname + " " + coltype + ","
    create_table_str = create_table_str[:-1] + ");"
    c.execute(create_table_str)
    
    csv_df = pd.read_csv(csv_file)
    for row in csv_df.iterrows():
        cmd_str = "INSERT OR IGNORE INTO stop_times(trip_id,arrival_time,stop_id,stop_sequence,stop_headsign,arrival_id) VALUES ("
        trip_id = row[1]['trip_id']
        arrival_time = row[1]['arrival_time']
        stop_id = row[1]['stop_id']
        stop_sequence = row[1]['stop_sequence']
        stop_headsign = row[1]['stop_headsign']
        arrival_id = trip_id + " " + arrival_time # unique arrival identifier
        cmd_str += "'{}', '{}', '{}', {}, '{}', '{}')".format(trip_id, arrival_time, stop_id, stop_sequence, stop_headsign, arrival_id)
        c.execute(cmd_str)
    
    conn.commit()
    conn.close()

# initial creation of database 
# cumtd_csv_to_sqlite('../../google_transit/stop_times.txt', 'stop_times', 'stop_times.db')

In [54]:
conn = sqlite3.connect('stop_times.db')
c = conn.cursor()

In [55]:
db_name = 'stop_times'
for bus in df.iterrows():
    diff = bus[1]['diff']
    trip_id = bus[1]['trip_id']
    arrival_time = bus[1]['scheduled_time'][11:19]
    arrival_date = bus[1]['scheduled_time'][:10]
    
    new_row = True
    for row in c.execute("PRAGMA table_info('{}')".format(db_name)):
        if row[1] == arrival_date:
            new_row = False

    if new_row:
        c.execute("ALTER TABLE {} ADD '{}' INTEGER(4) DEFAULT -1".format(db_name, arrival_date))

    
    exec_str = "UPDATE {} SET '{}' = {} WHERE arrival_id = '{}';".format(db_name, arrival_date, diff, trip_id + " " + arrival_time)
    c.execute(exec_str)
    
conn.commit()
conn.close()

In [56]:
# c = sqlite3.connect('stop_times.db').cursor()
# c.execute("SELECT * FROM stop_times WHERE \"2018-11-06\" IS NOT -1".format(arrival_date))
# c.fetchall()

In [57]:
import time, re

In [58]:
def update_db(arrival_date, diff, trip_id, arrival_time, db_file = 'stop_times.db', debug = True):
    db_name = db_file[:-3]
    conn = sqlite3.connect(db_file)
    c = conn.cursor()

    new_row = True
    for row in c.execute("PRAGMA table_info('{}')".format(db_name)):
        if row[1] == arrival_date:
            new_row = False
    if new_row:
        c.execute("ALTER TABLE {} ADD '{}' INTEGER(4) DEFAULT -1".format(db_name, arrival_date))

    exec_str = "UPDATE {} SET '{}' = {} WHERE arrival_id = '{}';".format(db_name, arrival_date, diff, trip_id + " " + arrival_time)
    c.execute(exec_str)

    conn.commit()
    conn.close()

In [59]:
def parse_store_cumtd_data(result, debug = True):
    if 'departures' not in result:
        if debug: print("{}: error getting departures: {}".format(datetime.now(), result['status']['msg']))
    else:
        for departure in result['departures']:
            if (departure['is_scheduled']):
                trip_id = departure['trip']['trip_id']
                scheduled_time = datetime.fromisoformat(departure['scheduled'])
                diff = (datetime.fromisoformat(departure['expected']) - scheduled_time).seconds
                arrival_date, arrival_time = str(scheduled_time).split(' ')
                if scheduled_time.hour <= 6:
                    arrival_date = str(scheduled_time - timedelta(1,0))[:10]
                    arrival_time = str(int(arrival_time[:2]) + 24) + arrival_time[2:-6]
                else:
                    arrival_time = arrival_time[:-6]
                update_db(arrival_date, diff, trip_id, arrival_time, debug = True)
            else:
                print("{}: unscheduled ride: {}".format(datetime.now(), result))
        if debug: print("{}: finished analyzing {}: {} departures logged".format(datetime.now(), result['rqst']['params']['stop_id'].upper(), len(result['departures'])))

In [60]:
def has_stops(stop_id, minutes, db_file = 'stop_times.db', debug = True):
    conn = sqlite3.connect(db_file)
    c = conn.cursor()
    for stop in c.execute("SELECT arrival_time FROM {0} WHERE stop_id LIKE '{1}:%' OR stop_id LIKE '{1}' ORDER BY arrival_time".format(db_file[:-3], stop_id)):
        other_time = [int(x) for x in stop[0].split(":")]
        if other_time[0] > 23: other_time[0] -= 24
        now_time = datetime.now().time()
        if timedelta(minutes=0) < timedelta(hours=other_time[0], minutes=other_time[1], seconds=other_time[2]) \
            - timedelta(hours=now_time.hour, minutes=now_time.minute, seconds=now_time.second) < timedelta(minutes=minutes):
            conn.close()
            if debug: print("{}: {} has a stop within next {} minutes".format(datetime.now(), stop_id, minutes))
            return True
    if debug: print("{}: {} does not have a stop within next {} minutes".format(datetime.now(), stop_id, minutes))
    conn.close()
    return False

In [64]:
def analyze_all_stops(stops, minutes_between, debug):
    start = datetime.now()
    stops_analyzed = 0
    for stop_id in stops:
        if has_stops(stop_id, minutes_between, 'stop_times.db', debug):
            r = requests.get(cumtdRequestUrl("getdeparturesbystop", {'stop_id': stop_id, 'pt': minutes_between}))
            parse_store_cumtd_data(r.json(), debug)
            stops_analyzed += 1
        time.sleep(3)
    print("{}: finished analyzing {} stops in {}".format(datetime.now(), stops_analyzed, datetime.now() - start))

In [ ]:
debug = True
minutes_between = 60


all_stops = stop_times_all.loc[:]['stop_id']
stops = set([x[:-2] if re.match(".+:\w{1}", x) else x for x in all_stops])
stops = sorted(stops)


while True:
    analyze_all_stops(stops, minutes_between, debug)

    while datetime.now().time().minute != 0:
        time.sleep(60)
    print("{}: starting next round".format(datetime.now()))

In [62]:
conn = sqlite3.connect("stop_times.db")
c = conn.cursor()
c.execute("PRAGMA table_info('stop_times')")
print(c.fetchall())
c.execute("SELECT * FROM stop_times WHERE arrival_id = '[@7.0.41950648@][1244056065453]/117__I5_UIMTH 26:28:40';")
c.fetchall()

[(0, 'trip_id', 'VARCHAR(60)', 1, None, 0), (1, 'arrival_time', 'VARCHAR(8)', 1, None, 0), (2, 'stop_id', 'VARCHAR(17)', 1, None, 0), (3, 'stop_sequence', 'INTEGER', 0, None, 0), (4, 'stop_headsign', 'VARCAHAR(36)', 1, None, 0), (5, 'arrival_id', 'VARCHAR(63)', 1, None, 1), (6, '2018-11-07', 'INTEGER(4)', 0, '-1', 0)]


[('[@7.0.41950648@][1244056065453]/117__I5_UIMTH',
  '26:28:40',
  '1STGRG:2',
  15,
  'nan',
  '[@7.0.41950648@][1244056065453]/117__I5_UIMTH 26:28:40',
  -1)]

In [63]:
c.execute("SELECT * FROM stop_times WHERE stop_id LIKE '1STARY:_'")
c.fetchall()

[('[@124.0.92263401@][1484328944432]/275__I1UISU',
  '18:26:39',
  '1STARY:4',
  20,
  'nan',
  '[@124.0.92263401@][1484328944432]/275__I1UISU 18:26:39',
  -1),
 ('[@124.0.92263401@][1484328944432]/179__I1UISU',
  '18:59:20',
  '1STARY:5',
  16,
  'nan',
  '[@124.0.92263401@][1484328944432]/179__I1UISU 18:59:20',
  -1),
 ('[@124.0.92263401@][1484328944432]/351__I1UISU',
  '19:36:39',
  '1STARY:4',
  20,
  'nan',
  '[@124.0.92263401@][1484328944432]/351__I1UISU 19:36:39',
  -1),
 ('[@124.0.92263401@][1484328944432]/92__I1UISU',
  '20:09:20',
  '1STARY:5',
  16,
  'nan',
  '[@124.0.92263401@][1484328944432]/92__I1UISU 20:09:20',
  -1),
 ('[@124.0.92263401@][1484328944432]/233__I1UISU',
  '20:46:39',
  '1STARY:4',
  20,
  'nan',
  '[@124.0.92263401@][1484328944432]/233__I1UISU 20:46:39',
  -1),
 ('[@124.0.92263401@][1484328944432]/96__I1UISU',
  '21:19:20',
  '1STARY:5',
  16,
  'nan',
  '[@124.0.92263401@][1484328944432]/96__I1UISU 21:19:20',
  -1),
 ('[@124.0.92263401@][1484328944432]/2

In [ ]:
conn.commit()
conn.close()